# Import Required Libraries
Import the necessary libraries, including LeagueHistoryLoader.

In [1]:
# LeagueHistoryLoader for loading the league history data
from src.fpl_load import LeagueHistoryLoader

# Setting the league id
league_id = 665568

# Creating an instance of LeagueHistoryLoader
history_loader = LeagueHistoryLoader(league_id)

In [28]:
# Loading the league history data
league_data = history_loader.get_data()

df = league_data


# Answer Initial Questions
Use the loaded data to answer the questions found in the Initial Questions Answered section in README.md.

In [49]:
df

,event,event_points,cumulative_points,fpl_event_rank,fpl_event_rank_sort,overall_rank,percentile_rank,bank,team_value,event_transfers,...,points_on_bench,entry,id,event_total,player_name,league_rank,last_rank,rank_sort,standings_total,entry_name
0,1,80,80,710070,784384,710070,NaN,0,1000,0,...,11,3110982,21377033,84,Dominic Rech,1,1,1,1482,In Good Kompany
1,2,52,132,2319915,2363147,812849,NaN,10,1002,1,...,21,3110982,21377033,84,Dominic Rech,1,1,1,1482,In Good Kompany
2,3,53,185,1862688,1894075,531104,NaN,6,1008,1,...,20,3110982,21377033,84,Dominic Rech,1,1,1,1482,In Good Kompany
3,4,64,249,7650408,7666623,1455389,NaN,19,1013,1,...,8,3110982,21377033,84,Dominic Rech,1,1,1,1482,In Good Kompany
4,5,38,287,7572009,7616547,2126814,NaN,17,1011,1,...,7,3110982,21377033,84,Dominic Rech,1,1,1,1482,In Good Kompany
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,20,100,1077,81475,87641,2321904,NaN,31,1017,1,...,0,5025791,39069622,82,Fin Whittington,16,16,16,1288,Is Your Motherwell
380,21,42,1115,6574103,6596978,2685966,65.0,122,1017,2,...,4,5025791,39069622,82,Fin Whittington,16,16,16,1288,Is Your Motherwell
381,22,49,1160,5413499,5430275,2945224,55.0,71,1017,2,...,1,5025791,39069622,82,Fin Whittington,16,16,16,1288,Is Your Motherwell
382,23,46,1206,7330969,7387422,3430132,70.0,18,1019,0,...,2,5025791,39069622,82,Fin Whittington,16,16,16,1288,Is Your Motherwell


In [44]:
import duckdb

# Convert DataFrame to DuckDB
duckdb_df = duckdb.from_df(df)

# mydf = pd.DataFrame({'a' : [1, 2, 3]})

# print(duckdb.query("SELECT SUM(a) FROM mydf").to_df())

# Most points left on bench in a week
total_points_left_on_bench = duckdb.query("""
    SELECT player_name, entry_name, SUM(points_on_bench) AS total_points_on_bench
    FROM duckdb_df
    GROUP BY player_name, entry_name
    ORDER BY total_points_on_bench DESC
""").to_df()

most_points_left_on_bench_week = duckdb.query("""
    SELECT d.player_name, d.entry_name, d.points_on_bench AS most_points_left_on_bench, d.event
    FROM duckdb_df d
    JOIN (
        SELECT player_name, entry_name, MAX(points_on_bench) AS max_points
        FROM duckdb_df
        GROUP BY player_name, entry_name
    ) m ON d.player_name = m.player_name AND d.entry_name = m.entry_name AND d.points_on_bench = m.max_points
    ORDER BY most_points_left_on_bench DESC
""").to_df()

In [45]:
total_points_left_on_bench

,player_name,entry_name,total_points_on_bench
0,Daniel Brown,Benson and Hedges,212.0
1,Jack Chalkley,Garnachos,204.0
2,Dominic Rech,In Good Kompany,202.0
3,Liam Aylwin,Unbelievable Jeff!!,201.0
4,Jack Grimley,Cancelo Culture,185.0
5,Mark Longmire,Botman Begins,182.0
6,Oliver Bignall,Biggie FC,177.0
7,Andrew Wallace,Boggie FC,172.0
8,Prince Ehigiator,McCris-P FC,157.0
9,Mason Scott,FC Mase,143.0


In [46]:
most_points_left_on_bench_week

,player_name,entry_name,most_points_left_on_bench,event
0,Daniel Brown,Benson and Hedges,28,6
1,Oliver Bignall,Biggie FC,27,18
2,Mark Longmire,Botman Begins,24,14
3,Patrick Prendergast,Who Got The Assist,24,23
4,Liam Aylwin,Unbelievable Jeff!!,22,21
5,Dominic Rech,In Good Kompany,21,2
6,Jack Chalkley,Garnachos,19,20
7,Mason Scott,FC Mase,19,5
8,Jack Grimley,Cancelo Culture,19,14
9,William Levick,Haalandaise Sauce,18,22


In [53]:
biggest_difference = duckdb.query("""
    SELECT a.event, a.player_name AS player1, a.entry_name AS entry1, a.event_points AS points1,
           b.player_name AS player2, b.entry_name AS entry2, b.event_points AS points2,
           ABS(a.event_points - b.event_points) AS difference
    FROM duckdb_df a, duckdb_df b
    WHERE a.event = b.event AND a.player_name != b.player_name
    ORDER BY difference DESC
    LIMIT 1
""").to_df()
biggest_difference

,event,player1,entry1,points1,player2,entry2,points2,difference
0,23,Jack Grimley,Cancelo Culture,44,William Levick,Haalandaise Sauce,110,66


In [6]:
# Most points left on bench in a week
most_points_left_on_bench = df['points_on_bench'].max()
print(f"Most points left on bench: {most_points_left_on_bench}")


Most points left on bench: 28


In [ ]:

# Worst week
worst_week = df['event_total_points'].idxmin()
print(f"Worst week: {worst_week}")


In [ ]:

# Biggest difference in points on a week from winner and loser
df_grouped = df.groupby('event')['event_total_points'].agg(['min', 'max'])
df_grouped['difference'] = df_grouped['max'] - df_grouped['min']
biggest_difference = df_grouped['difference'].max()
print(f"Biggest difference in points on a week from winner and loser: {biggest_difference}")